# Roadmap

$$ \text{Raster}\xleftarrow{\text{1ms bin}} \text{Relative Response: } T \times (N * B)$$
<br>
$$\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\downarrow{\frac{\Sigma{\text{ (columns: T)}}} {\text{T in E}}} $$
<br>
$$ PSTH $$
<br>
$$\text{T = trials | N = neurons | B = bins | E = events}$$

# Goal
* Find neurons' relative response to each trial
* Create the relative response matrix: Trials×(Neurons*Total Bins)
  * Create raster plot if bin size = 1ms
  * Sum columns and create PSTH
    * Plot PSTHS
    * Receptive field analysis if bin size = 5ms

# JSON Example
## Human Friendly Version (Pretty format)
```json
{
    "events": {
        "event_1": [
            138.33675
        ],
         "event_4": [
            42.74935
        ]
    }
}
```

## Computer Friendly Version
```json
{"events": {"event_1":[138.33675],"event_4": [42.74935]}}
```

# JSON Files
## Matlab
Note: comments use # for syntax highlighting instead of Matlab's  %
```py
# get file name and path
# open file with fopen and load in data
# close file and use jsondecode on the data read in from file
```

```py
# jsondecode reads str_data into a struct
example_data = jsondecode(str_data);
neuron_names = fieldnames(example_data.neurons)
for neuron_i = 1:length(neuron_names)
    # Get current neuron fieldname
    curr_neuron = neuron_names{neuron_i};
    # Get timestamps stored in struct
    neuron_ts = example_data.neurons.(curr_neuron);
end
```

# JSON Files
## Python

```py
import json
# open file
# Use load or other json functions to read in data
example_data = json.load(file)
```

In [2]:
import json

with open('hw1_example.json') as input:
    example_data = json.load(input)

In [3]:
    print(example_data.keys())
    print(example_data['events'].keys())
    print(example_data['neurons'].keys())

dict_keys(['events', 'neurons'])
dict_keys(['event_1', 'event_4'])
dict_keys(['sig001a', 'sig007c'])


# Psuedocode: Creating Relative Response
```py
# Go through collection of bin sizes to create PSTH
for bin_size in bin_collection:
    window_range = pre_time:bin_size:post_time
```

```py
# Go through event keys and grab event timestamps
    for curr_event in events:
        event_ts = example_data['events'][curr_event]
```

```py
# Go through neurons and grab neuron spike times
        for curr_neuron in neurons:
            neuron_ts = example_data['neurons'][curr_neuron]
```

```py
# Go through trials and grab trial timestamp to create histogram
            for curr_trial in curr_event:
                # Create offset time window so that event onset is at t = 0
                offset_ts = neuron_ts - curr_trial
                # bin spikes within analysis window
                binned_response = histogram(offset_ts, window_range)
                # Populate relative response matrix with neuron response
                response[trial_i, unit_start:unit_end] = binned_response
```

```py
            # Create PSTH for each neuron for each event
            psth = sum(response) / tot_event_trials
```

# Example PSTH
![Sig 007c Event 4 PSTH](images/sig007c_event_4_0.01ms_psth.png)

# Psuedocode: Receptive Field Analysis
```py
for curr_event in events:
    for curr_neuron in neurons:
        if bin_size == 0.005:
            % Split psth into pre and post windows
            pre_time_psth, post_time_psth = split_psth(neuron_psth)
```

```py
            # Pre time Receptive field
            # If using numpy.std, set ddof = 1
            threshold = mean(pre_time_psth) + 3*std(pre_time_psth)
            # Threshold is arbitrary
```

```py
            # Post time receptive field
            above_threshold_bins = post_time > threshold
            # Note: Dont forget to scale bin indices into time
            first_bin = above_threshold[0]
            last_bin = above_threshold[-1]
            peak = max(above_threshold)
            magnitude = sum(above_threshold) # grab bin magnitudes
```

# Example PSTH with Receptive Field
![Sig 007c Event 4 PSTH](images/sig007c_event_4_0.005ms_psth.png)

# What to turn in
## Conceptual Questions and graphs (10pts)
* pdf with answers to conceptual questions and requested psths and raster graphs
* Code files you wrote to create JSON output file
* Code files should be plain text with proper file extension
  * ex: last_first_hw1.m, last_first_hw1.py, etc.

# Json Output File (10pts)
* This file will be input into automatic grader
* This was taken from the example solution json and is a small snippet
* You are not required to turn in a pretty formatted json file
```json
{
    "event_1": {
        "sig001a": {
            "background_rate": 0.013732394366197184,
            "first_bin_latency": 0.0925,
            "last_bin_latency": 0.1325
        }
    }
}
```

# Python Raster Sidenote
## Example
Parameters: 1ms bin size from 0ms to 5ms
$$\text{0 0 1 0 1} \xrightarrow{\text{transformation}} \text{NaN NaN .003 NaN .005}$$
### Mapping
$$0 \rightarrow NaN$$
$$1 \rightarrow \text{scaled time point}$$

# Sig007c Event 4 Raster
Y-axis: trials X-axis: Time (s)
![Sig 007c Event 4 Raster](images/sig007c_event_4_0.001ms_raster.png)